In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import unicodedata

spark = SparkSession.builder.getOrCreate()

df = spark.table("projeto_final_bronze.bronze_panorama_eja")
df = df.toPandas()

df.head()

## Ajustando colunas

In [0]:
new_column_names = [
    'regiao', 'uf', 'municipio',
    'articulador_regional_undime_consed', 'formador_regional_undime', 'formador_regional_consed',
    'codigo_ibge', 'adesao_pacto_simec', 'taxa_analfabetismo_censo_ibge_2022',
    'populacao_total_municipio_15_anos_censo_ibge_2022', 'populacao_nao_alfabetizada_censo_ibge_2022',
    'matriculas_eja_fundamental_medio', 'matriculas_eja_ensino_fundamental', 'matriculas_eja_ensino_medio',
    'escolas_municipais_eja', 'escolas_estaduais_eja', 'tem_tipo_matricula_eja',
    'aderiu_pba_2024_sba', 'aderiu_pba_2025_sba', 'aderiu_pba_saldos_simec', 'adesao_pe_de_meia',
    'unidade_prisional_oferta_educacional', 'total_geral_estabelecimentos_por_nivel',
    'total_estab_fund', 'estab_fund_fed', 'estab_fund_est', 'estab_fund_mun', 'estab_fund_privado',
    'total_estab_medio', 'estab_medio_fed', 'estab_medio_est', 'estab_medio_mun', 'estab_medio_privado',
    'total_geral_estabelecimentos_por_localizacao', 'total_estab_urbano', 'estab_urbano_fed',
    'estab_urbano_est', 'estab_urbano_mun', 'estab_urbano_privado', 'total_estab_rural',
    'estab_rural_fed', 'estab_rural_est', 'estab_rural_mun', 'estab_rural_privado',
    'total_geral_matriculas_por_nivel', 'total_matricula_fund', 'mat_fund_fed', 'mat_fund_est',
    'mat_fund_mun', 'mat_fund_privado', 'total_matricula_medio', 'mat_medio_fed', 'mat_medio_est',
    'mat_medio_mun', 'mat_medio_privado', 'total_matriculas_medio_tecnico_integrado',
    'mat_medio_tecnico_integrado_fed', 'mat_medio_tecnico_integrado_est', 'mat_medio_tecnico_integrado_mun',
    'mat_medio_tecnico_integrado_privado', 'total_matriculas_medio_fic_integrado',
    'mat_medio_fic_integrado_fed', 'mat_medio_fic_integrado_est', 'mat_medio_fic_integrado_mun',
    'mat_medio_fic_integrado_privado', 'total_matriculas_fund_fic_integrado',
    'mat_fund_fic_integrado_fed', 'mat_fund_fic_integrado_est', 'mat_fund_fic_integrado_mun',
    'mat_fund_fic_integrado_privado', 'total_geral_matriculas_ed_basica_localizacao',
    'mat_total_ed_basica_urbana', 'mat_fed_ed_basica_urbana', 'mat_est_ed_basica_urbana',
    'mat_mun_ed_basica_urbana', 'mat_priv_ed_basica_urbana', 'mat_total_ed_basica_rural',
    'mat_fed_ed_basica_rural', 'mat_est_ed_basica_rural', 'mat_mun_ed_basica_rural', 'mat_priv_ed_basica_rural',
    'total_geral_matriculas_ed_basica_sexo_raca', 'total_feminino_matriculas_ed_basica',
    'total_feminino_matriculas_ed_basica_nao_declarada', 'total_feminino_matriculas_branca',
    'total_feminino_matriculas_preta', 'total_feminino_matriculas_ed_basica_parda',
    'total_feminino_matriculas_ed_basica_amarela', 'total_feminino_matriculas_ed_basica_indigena',
    'total_masculino_matriculas_ed_basica_nao_declarada', 'total_masc_matriculas_ed_basica_nao_declarada',
    'total_masc_matriculas_ed_basica_branca', 'total_masc_matriculas_ed_basica_preta',
    'total_masc_matriculas_ed_basica_parda', 'total_feminino_matriculas_ed_basica_amarela_1',
    'total_feminino_matriculas_ed_basica_indigena_1', 'total_geral_matriculas_ed_basica_faixa_etaria',
    'total_ate_14_anos_matriculas_ed_basica', 'total_ate_14_anos_matriculas_ed_basica_1',
    'total_entre_18_19_anos_matriculas_ed_basica', 'total_entre_20_24_anos_matriculas_ed_basica',
    'total_entre_25_29_anos_matriculas_ed_basica', 'total_entre_30_34_anos_matriculas_ed_basica',
    'total_entre_35_39_anos_matriculas_ed_basica', 'total_entre_40_anos_ou_matriculas_ed_basica',
    'total_geral_matriculas_ed_basica_ed_especial', 'total_geral_matriculas_ens_fund_ed_especial',
    'total_geral_matriculas_ens_medio_ed_especial', 'total_geral_matriculas_ed_basica_classes_exc',
    'total_geral_matriculas_ens_fund_classes_exc', 'total_geral_matriculas_ens_medio_classes_exc',
    'total_geral_matriculas_ed_basica_ed_indigena', 'total_geral_matriculas_ens_fund_ed_indigena',
    'total_geral_matriculas_ens_medio_ed_indigena', 'total_geral_docentes_ed_basica',
    'total_geral_docentes_ens_fund', 'total_geral_docentes_ens_fund_publica',
    'total_geral_docentes_ens_fund_federal', 'total_geral_docentes_ens_fund_estadual',
    'total_geral_docentes_ens_fund_municipal', 'total_geral_docentes_ens_fund_privada',
    'total_geral_docentes_ens_fund_1', 'total_geral_docentes_ens_fund_publica_1',
    'total_geral_docentes_ens_fund_federal_1', 'total_geral_docentes_ens_fund_estadual_1',
    'total_geral_docentes_ens_fund_municipal_1', 'total_geral_docentes_ens_fund_privada_1',
    '15_19_anos_populacao_nao_alfabetizada', '20_24_anos_populacao_nao_alfabetizada',
    '25_34_anos_populacao_nao_alfabetizada', '35_44_anos_populacao_nao_alfabetizada',
    '45_54_anos_populacao_nao_alfabetizada', '55_64_anos_populacao_nao_alfabetizada',
    '65_anos_ou_mais_populacao_nao_alfabetizada', 'unnamed_134', 'unnamed_135',
    'source_file', 'ingestion_time'
]

df.columns = new_column_names


In [0]:
df_empty = df[df.isna().all(axis=1)]
print(df_empty)

In [0]:
# verificando nulos
df_with_nulls = df[df.isna().any(axis=1)]

print(df_with_nulls)
print(f"\nTotal de linhas com algum valor nulo: {len(df_with_nulls)}")

## Alterando tipos de dados

In [0]:
exclude = ['regiao', 'uf', 'municipio', 'adesao_pacto_simec', 
           'tem_tipo_matricula_eja', 'aderiu_pba_2024_sba', 'aderiu_pba_2025_sba',
           'aderiu_pba_saldos_simec', 'adesao_pe_de_meia', 'unidade_prisional_oferta_educacional']

# converter todas as outras para float
cols_to_convert = [c for c in df.columns if c not in exclude]
for c in cols_to_convert:
    df[c] = pd.to_numeric(df[c], errors='coerce')

## Validação

In [0]:
null_counts = df.isna().sum()

null_counts = null_counts[null_counts > 0].sort_values(ascending=False)
print(null_counts)

In [0]:
# Lista de colunas a remover  (+ de 5 mil valores nulos)
colunas_para_remover = [
    'source_file',
    'formador_regional_undime',
    'formador_regional_consed',
    'unnamed_135',
    'unnamed_134',
    'articulador_regional_undime_consed'
]

# Remove as colunas
df = df.drop(columns=colunas_para_remover)

# Verifica se realmente foram removidas
print(df.columns)

In [0]:
# Filtra linhas com pelo menos um valor ausente
df_missing_rows = df[df.isna().any(axis=1)]

# Mostra as primeiras linhas para ter uma ideia
print(df_missing_rows.head())

# Quantas linhas têm algum valor ausente
print(f"Total de linhas com algum valor ausente: {len(df_missing_rows)}")

In [0]:
# Verificar se há linhas duplicadas
duplicados = df.duplicated()

# Quantas linhas duplicadas existem
print("Total de linhas duplicadas:", duplicados.sum())

# Mostrar as linhas duplicadas
df_duplicados = df[duplicados]
print(df_duplicados)

In [0]:
# excluindo duplicadas 
df = df.drop_duplicates()

In [0]:
# verificando valores unicos
unique_counts = df.nunique()
unique_counts_sorted = unique_counts.sort_values()
print(unique_counts_sorted)


In [0]:
df.describe()

In [0]:
df.isna().all(axis=1).sum()


### Ajustando coluna adesão

In [0]:
df = df[df['adesao_pacto_simec'] != 'nan']

print(df.shape)

In [0]:
df['adesao_pacto_simec'].unique()

In [0]:
# extraindo o ano
def extrair_ano(valor):
    if pd.isna(valor):
        return None
    elif 'não aderiu' in valor.lower():
        return 'não'
    else:
        ano = ''.join([c for c in valor if c.isdigit()])
        return ano
    

df['ano_adesao'] = df['adesao_pacto_simec'].apply(extrair_ano)
df['ano_adesao'].unique()
df.drop(['adesao_pacto_simec'], axis=1, inplace=True)

In [0]:
df['ano_adesao'].unique()

# Agrupando colunas

In [0]:
# Matrículas por nível
df['matriculas_fund_total'] = df[['mat_fund_fed', 'mat_fund_est', 'mat_fund_mun', 'mat_fund_privado']].sum(axis=1)
df['matriculas_medio_total'] = df[['mat_medio_fed', 'mat_medio_est', 'mat_medio_mun', 'mat_medio_privado']].sum(axis=1)


In [0]:
# Matrículas urbano x rural
df['matriculas_urbanas_total'] = df[['mat_fed_ed_basica_urbana', 'mat_est_ed_basica_urbana', 
                                     'mat_mun_ed_basica_urbana', 'mat_priv_ed_basica_urbana']].sum(axis=1)
df['matriculas_rurais_total'] = df[['mat_fed_ed_basica_rural', 'mat_est_ed_basica_rural', 
                                    'mat_mun_ed_basica_rural', 'mat_priv_ed_basica_rural']].sum(axis=1)

In [0]:
# Docentes por nível
df['docentes_fund_total'] = df[['total_geral_docentes_ens_fund_federal',
                                'total_geral_docentes_ens_fund_estadual',
                                'total_geral_docentes_ens_fund_municipal',
                                'total_geral_docentes_ens_fund_privada']].sum(axis=1)

In [0]:
colunas_para_excluir = [
    'mat_fund_fed', 'mat_fund_est', 'mat_fund_mun', 'mat_fund_privado',
    'mat_medio_fed', 'mat_medio_est', 'mat_medio_mun', 'mat_medio_privado',
    'mat_fed_ed_basica_urbana', 'mat_est_ed_basica_urbana', 'mat_mun_ed_basica_urbana', 'mat_priv_ed_basica_urbana',
    'mat_fed_ed_basica_rural', 'mat_est_ed_basica_rural', 'mat_mun_ed_basica_rural', 'mat_priv_ed_basica_rural',
    'total_geral_docentes_ens_fund_federal', 'total_geral_docentes_ens_fund_estadual',
    'total_geral_docentes_ens_fund_municipal', 'total_geral_docentes_ens_fund_privada',
    'total_geral_matriculas_ed_basica_classes_exc', 'total_geral_matriculas_ens_fund_classes_exc',
    'total_geral_matriculas_ens_medio_classes_exc', 'total_geral_matriculas_ens_medio_ed_especial',
    'adesao_pe_de_meia'
]

df = df.drop(columns=colunas_para_excluir)

print(df.head())

In [0]:
colunas_matriculas = ['matriculas_fund_total','matriculas_medio_total','matriculas_urbanas_total','matriculas_rurais_total',
                          'total_entre_18_19_anos_matriculas_ed_basica', 'total_entre_20_24_anos_matriculas_ed_basica',
                            'total_entre_25_29_anos_matriculas_ed_basica', 'total_entre_30_34_anos_matriculas_ed_basica',
                            'total_entre_35_39_anos_matriculas_ed_basica', 'total_entre_40_anos_ou_matriculas_ed_basica',]
df[colunas_matriculas] = df[colunas_matriculas].fillna(0).astype(int)


In [0]:
df['populacao_total_municipio_15_anos_censo_ibge_2022'] = df['populacao_total_municipio_15_anos_censo_ibge_2022'].fillna(0).astype(int)
df['populacao_nao_alfabetizada_censo_ibge_2022'] = df['populacao_nao_alfabetizada_censo_ibge_2022'].fillna(0).astype(int)


In [0]:

df['taxa_analfabetismo_censo_ibge_2022'] = df['taxa_analfabetismo_censo_ibge_2022'].astype(float)

In [0]:
# Colunas de adesão / programas
colunas_adesao = [
    'aderiu_pba_2024_sba', 
    'aderiu_pba_2025_sba', 
    'aderiu_pba_saldos_simec', 
]

# Verificar valores únicos em cada coluna
for col in colunas_adesao:
    print(f"Valores únicos em {col}: {df[col].unique()}")
    print(df[col].value_counts(dropna=False))
    print("-"*50)

In [0]:
# Padronizar adesão 2024
df['aderiu_pba_2024_sba'] = df['aderiu_pba_2024_sba'].str.strip().map({'SIM': 1, 'Não': 0}).astype(int)

# Padronizar adesão 2025
df['aderiu_pba_2025_sba'] = df['aderiu_pba_2025_sba'].str.strip().map({'SIM': 1, 'Não': 0}).astype(int)

# Padronizar saldos simec
df['aderiu_pba_saldos_simec'] = df['aderiu_pba_saldos_simec'].str.strip().map({'SIM': 1, 'Não': 0}).astype(int)


In [0]:
df.head()

In [0]:
# Removendo acentos da coluna UF
df["uf"] = df["uf"].apply(
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8') if isinstance(x, str) else x
)

In [0]:
spark.sql("DROP TABLE IF EXISTS projeto_final_silver.silver_panorama_eja")

In [0]:
df_spark = spark.createDataFrame(df)

# Salva como tabela Delta no schema Silver
df_spark.write.format("delta") \
       .mode("overwrite") \
       .saveAsTable("projeto_final_silver.silver_panorama_eja")